In [ ]:
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import re
import random

from IPython.display import display

from nltk import word_tokenize, sent_tokenize

from sklearn.model_selection import train_test_split, cross_val_score, GridSearchCV,StratifiedKFold
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.naive_bayes import MultinomialNB
from sklearn.metrics import confusion_matrix, precision_recall_fscore_support, classification_report
from sklearn.dummy import DummyClassifier
import nltk

In [ ]:
train_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_train_preprocesado.csv'
df_train = pd.read_csv(train_url)
x_train = df_train['review_es']
y_train = df_train['sentimiento']
test_url = 'https://media.githubusercontent.com/media/ricadolcvuba/Dataset-TP2/main/df_test_preprocesado.csv'
df_test = pd.read_csv(test_url)
x_test = df_test['review_es']

#Naive Bayes

##Vectorizar

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer
import nltk
from nltk.corpus import stopwords
from nltk import word_tokenize
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


True

In [ ]:
x_train_text = df_train['review_es']
y_train_text = df_train['sentimiento']

x_test_text = df_test['review_es']

In [ ]:
my_stop_words = stopwords.words('spanish')

# Instanciamos el CV
#Configurar el vectorizador de palabras para convertir el texto en una representación numérica.
vec = CountVectorizer(stop_words=my_stop_words, ngram_range=(1,2))

# Ajustamos el CV y transformamos los datos
#transformar los datos de entrenamiento en una matriz numérica.
x_train_aux = vec.fit_transform(x_train_text)
#Transformar los datos de prueba en la misma representación numérica que el conjunto de entrenamiento
x_test = vec.transform(x_test_text)

In [ ]:
print(x_train_aux)

  (0, 697421)	1
  (0, 1898993)	1
  (0, 830041)	1
  (0, 3079904)	2
  (0, 2789740)	2
  (0, 2151486)	6
  (0, 1043420)	2
  (0, 1119292)	1
  (0, 1016518)	1
  (0, 2488725)	1
  (0, 1149226)	1
  (0, 2838667)	1
  (0, 614205)	1
  (0, 2395540)	2
  (0, 668127)	1
  (0, 1332604)	1
  (0, 375201)	1
  (0, 1058107)	1
  (0, 3131887)	4
  (0, 1545805)	1
  (0, 1001713)	1
  (0, 793434)	1
  (0, 2161246)	2
  (0, 607000)	1
  (0, 1093226)	4
  :	:
  (48178, 1045280)	1
  (48178, 1575866)	1
  (48178, 610347)	1
  (48178, 1597454)	1
  (48178, 3097086)	1
  (48178, 212858)	1
  (48178, 1890374)	1
  (48178, 1045281)	1
  (48178, 2997403)	1
  (48178, 1575867)	1
  (48178, 610348)	1
  (48178, 1000964)	1
  (48178, 2267815)	1
  (48178, 2080369)	1
  (48178, 87040)	1
  (48178, 2297534)	1
  (48178, 1597455)	1
  (48178, 1060247)	1
  (48178, 1334177)	1
  (48178, 1878490)	1
  (48178, 3184579)	1
  (48178, 2259041)	1
  (48178, 3097087)	1
  (48178, 1258685)	1
  (48178, 398987)	1


##Modelo

In [ ]:
from sklearn.naive_bayes import MultinomialNB
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.model_selection import StratifiedKFold, KFold,RandomizedSearchCV
from sklearn.metrics import make_scorer, f1_score

In [ ]:
#Cantidad de combinaciones que quiero porbar
n=10
#Conjunto de parámetros que quiero usar
params = {'alpha':np.linspace(0,1.5,15),
               'force_alpha':[True,False],
               'fit_prior':[True,False],
               'class_prior':[None, [0.2, 0.8], [0.5, 0.5]]}
#Cantidad de splits para el Cross Validation
folds=10
#Kfold
kfoldcv = KFold(n_splits=folds)
#Clasificador
nb = MultinomialNB()
#Metrica que quiero optimizar F1 Score
scorer_fn = make_scorer(f1_score, average='macro')
#Random Search Cross Validation
randomcv = RandomizedSearchCV(nb,
                              param_distributions = params,
                              scoring=scorer_fn,
                              cv=kfoldcv,
                              random_state=20,
                              n_iter=n)
#Busco los hiperparamtros que optimizan F1 Score
randomcv.fit(x_train_aux, y_train_text);

In [ ]:
#Mejores hiperparametros
print(randomcv.best_params_)
#Mejor métrica
print(randomcv.best_score_)

{'force_alpha': True, 'fit_prior': False, 'class_prior': [0.2, 0.8], 'alpha': 1.0714285714285714}
0.8754148480901254


In [ ]:
#Mejor estimador
best_nb=randomcv.best_estimator_

#Predicción con mejor estimador
y_pred_nb=best_nb.predict(x_test)

Crear dataset de predicciones

In [ ]:
df_pred_nb = pd.DataFrame()
df_pred_nb['ID'] = df_test['ID']
df_pred_nb['sentimiento'] = y_pred_nb

Descargar dataset

In [ ]:
df_pred_nb.to_csv('df_pred_nb.csv', index=False)

In [ ]:
import joblib

In [ ]:
joblib.dump(best_nb,'best_nb.joblib')

['best_nb.joblib']